In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

2025-08-04 12:50:59.804448: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754311860.069031      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754311860.149693      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Load data
df = pd.read_csv("/kaggle/input/515k-hotel-reviews-data-in-europe/Hotel_Reviews.csv")
df = df[['Negative_Review','Positive_Review']]
df.head()

,Negative_Review,Positive_Review
0,I am so angry that i made this post available...,Only the park outside of the hotel was beauti...
1,No Negative,No real complaints the hotel was great great ...
2,Rooms are nice but for elderly a bit difficul...,Location was good and staff were ok It is cut...
3,My room was dirty and I was afraid to walk ba...,Great location in nice surroundings the bar a...
4,You When I booked with your company on line y...,Amazing location and building Romantic setting


In [4]:
# Combine positive and negative into single dataset
pos = df[['Positive_Review']].copy()
pos['label'] = 1
pos.rename(columns={'Positive_Review': 'review'}, inplace=True)


In [5]:
neg = df[['Negative_Review']].copy()
neg['label'] = 0
neg.rename(columns={'Negative_Review': 'review'}, inplace=True)


In [6]:
data = pd.concat([pos, neg])
data = data[data['review'].str.lower() != 'no negative']  # Clean empty negative labels
data.dropna(inplace=True)

In [7]:
# Clean text
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in stop_words]
    return " ".join(tokens)

data['cleaned'] = data['review'].apply(clean_text)

In [8]:
# Tokenize
MAX_VOCAB = 10000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<OOV>")
tokenizer.fit_on_texts(data['cleaned'])
sequences = tokenizer.texts_to_sequences(data['cleaned'])
padded = pad_sequences(sequences, maxlen=MAX_LEN, padding='post', truncating='post')

In [9]:
# Prepare data
X = padded
y = data['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Build model
model = Sequential()
model.add(Embedding(input_dim=MAX_VOCAB, output_dim=128, input_length=MAX_LEN))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.build(input_shape=(None, MAX_LEN))
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-08-04 12:51:55.925880: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,333,633 (5.09 MB)

 Trainable params: 1,333,633 (5.09 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
# Train
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)




Epoch 1/10
18072/18072 ━━━━━━━━━━━━━━━━━━━━ 932s 51ms/step - accuracy: 0.5685 - loss: 0.6851 - val_accuracy: 0.5720 - val_loss: 0.6827
Epoch 2/10
18072/18072 ━━━━━━━━━━━━━━━━━━━━ 906s 50ms/step - accuracy: 0.6370 - loss: 0.5947 - val_accuracy: 0.9361 - val_loss: 0.1702
Epoch 3/10
18072/18072 ━━━━━━━━━━━━━━━━━━━━ 914s 51ms/step - accuracy: 0.9393 - loss: 0.1658 - val_accuracy: 0.9402 - val_loss: 0.1665
Epoch 4/10
18072/18072 ━━━━━━━━━━━━━━━━━━━━ 922s 51ms/step - accuracy: 0.9462 - loss: 0.1488 - val_accuracy: 0.9409 - val_loss: 0.1664
Epoch 5/10
18072/18072 ━━━━━━━━━━━━━━━━━━━━ 975s 54ms/step - accuracy: 0.9498 - loss: 0.1382 - val_accuracy: 0.9394 - val_loss: 0.1769
Epoch 6/10
18072/18072 ━━━━━━━━━━━━━━━━━━━━ 929s 51ms/step - accuracy: 0.9534 - loss: 0.1294 - val_accuracy: 0.9399 - val_loss: 0.1682
Epoch 7/10
18072/18072 ━━━━━━━━━━━━━━━━━━━━ 924s 51ms/step - accuracy: 0.9562 - loss: 0.1225 - val_accuracy: 0.9395 - val_loss: 0.1728
Epoch 8/10
18072/18072 ━━━━━━━━━━━━━━━━━━━━ 924s 51ms/s

In [13]:
# Evaluate
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("\nAccuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

5648/5648 ━━━━━━━━━━━━━━━━━━━━ 93s 16ms/step

Accuracy: 0.9386336723513983
              precision    recall  f1-score   support

           0       0.93      0.93      0.93     77609
           1       0.95      0.95      0.95    103109

    accuracy                           0.94    180718
   macro avg       0.94      0.94      0.94    180718
weighted avg       0.94      0.94      0.94    180718

